# Mask Scorer

## Description

This script calculates performance scores that measure the accuracy of a system output mask to a reference mask. The script generates CSV report tables, one containing scores for each mask and another containing an average of the scores in the first CSV. Given the option, the script will generate a detailed HTML index file for the mask region performance results.

In the following metrics, the following terminology is used:
 * $GT$ refers to the ground truth mask
 * $sys$ refers to the system output mask
 * $TP$ refers to True Positives computed between the ground truth mask and the system output
 * $TN$ refers to True Negatives computed between the ground truth mask and the system output
 * $FN$ refers to False Negatives computed between the ground truth mask and the system output
 * $FP$ refers to False Positives computed between the ground truth mask and the system output
 * $weights$ is a matrix of 1's and 0's that denotes the set of pixels scored by the system; it is generated by a difference in the erosion and dilation of the manipulated area of $GT$.

The following metrics are used to score each mask:

### Nimble Mask Metric (NMM)
\begin{equation*}
NMM(GT,sys,weights,c)=\max{\left(\frac{TP - FN - FP}{\Sigma_{px\in GT}weights(px)},c\right)}
\end{equation*}

$\Sigma_{px \in GT}$ refers to the sum over the pixels in the ground truth that are marked black. $c$ denotes a minimum cutoff value for the scoring to have any meaning; by default, $c=-1$.

### Matthews Correlation Coefficient (MCC)
\begin{equation*}
MCC(GT,sys) = \frac{TP*TN - FP*FN}{\sqrt{(TP+FP)(TP+FN)(TN+FP)(TN+FN)}}
\end{equation*}

An MCC of 1 denotes perfect correlation, an MCC of 0 denotes no correlation at all, and an MCC of -1 denotes perfect anti-correlation.

### Weighted L1 Loss (WL1)
\begin{equation*}
WL1(GT,sys,weights)=\frac{(FP+FN)_{weights > 0}}{\Sigma weights(px)}
\end{equation*}

A Weighted L1 of 0 denotes perfect or near perfect match up to variation within the weights that are 0; 1 denotes perfect mismatch. $(FP+FN)_{weights > 0}$ refers to the total number of $FP$ and $FN$ pixels where weights are greater than 0.

## Command-line Options

Example:
```
python2 MaskScorer.py -t removal --refDir ../../data/test_suite/maskScorerTests/ -r reference/removal/NC2016-removal-ref.csv -x index/NC2016-removal-index.csv -s ../../data/test_suite/maskScorerTests/B_NC2016_Removal_ImgOnly_c-me2_2/B_NC2016_Removal_ImgOnly_c-me2_2.csv -oR maskoutputs/sample -html
```
Running this code would produce, under the maskoutputs directory, an aggregate report of the computed mask scores titled sample.csv and a per-image score report titled sample-perimage.csv for the removal task. The -html flag is also set, allowing the code to generate an HTML per-image index file with the scores and metadata containing links to individual detailed reports of each image (described in the Sample HTML Output at the bottom of the page).

The command-line options for the mask scorer can be categorized as follows:

### Task Type Options

-t --task [manipulation, removal, splice, clone]

  * Specify the task type for evaluation (default = manipulation)

### Input Options

All CSV files passed to the Mask Scorer must contain headers and must have their rows separated by pipe characters ('|'). Fields and values in the CSV should <i>not</i> be enclosed in quotes ( ' or " ) if possible (e.g. entries 'foo', an empty field, and 'bar', in that order, should look like this in the csv: foo||bar). Additional specifications for the index and system output files can be found in the ValidatorNotebook.html file under the Validator directory.

--refDir

  * Specify the reference and index data path (e.g. "/NC2016_Test0601") (default = .)


-r --inRef

  * Specify the reference CSV file within refDir that contains the ground-truth information and metadata about each image. Key fields are TaskID, ProbeFileID, ProbeFileName, and ProbeMaskFileName, and if scoring on the 'splice' task, DonorFileID, DonorFileName, and DonorMaskFileName as well. Often the File ID's for the Probe and Donor will be the same as the file names, minus the extension. Additional fields, especially metadata pertaining to the ground-truth manipulation, may be included.

-x --inIndex

  * Define the index CSV file within refDir. The index file contains the TaskID, ProbeFileID, ProbeFileName, ProbeWidth, and ProbeHeight fields, and if scoring on the splice task, the DonorFileID, DonorFileName, DonorWidth, and DonorHeight fields as well. No additional fields are permitted for the index file.

--sysDir

  * Specify the system output data path, for example "mysysoutput/" (default = .) 


-s --inSys

  * Specify the CSV file of the system performance results formatted according to NC2016 specification. The file must contain the ProbeFileID, ConfidenceScore, and ProbeOutputMaskFileName fields, in that order, and if scoring on the splice task, the ProbeFileID, DonorFileID, ConfidenceScore, ProbeOutputMaskFileName, and DonorOutputMaskFileName fields, in that order. The ProbeOutputMaskFileNames and DonorOutputMaskFileNames (where relevant) should be directory strings relative to the location of the system performance CSV.

--rbin

  * Binarize the reference mask to black and white with a numeric threshold in the interval [0,255]. Pick -1 to not binarize and leave the mask as is. (default = 254)

--sbin

  * Binarize the system output mask to black and white with a numeric threshold in the interval [0,255]. Pick -1 to not binarize and leave the mask as is. (default = -1)

### Output Options

--outRoot

  * Specify the report output path and the file name suffix for saving the plot and table (e.g., test/sys_xxx). For example, if you specify "--outRoot test/NIST_001", you will find the aggregate score report "NIST_001.csv" and the per-image report "NIST_001-perimage.csv" in the "test" folder.


### Kernel Size Options

--eks

  * Erosion kernel size. (number must be odd; default = 15)
  
--dks

  * Dilation kernel size. (number must be odd; default = 9)

### Report Options

-v verbose

  * Control print output. Select 1 to print all non-error related output and 0 to suppress all print output (bar argument-parsing errors).
  
--precision

  * The number of digits to round computed scores, (e.g. a score of 0.3333333333333... will round to 0.33333 for a precision of 5), (default = 5).

-html

  * Output the report to HTML files. Set the flag to choose this option.

## Sample HTML Output

The following set of images is a sample of what you should see in the HTML page for each image. You can access each image from the appropriate link in the index.html file that is produced in your chosen output directory.

<h2>Manipulated Image: morelight.jpg</h2><br />
<img src="notebookImgs/morelight.jpg" alt="manipulated image" style="width:640px;">
<br />
<h3>Composite with Color Mask: sys-light_composite.jpg</h3>
<img src="notebookImgs/sys-light_composite.jpg" alt="manip image plus mask" style="width:640px;">
<br />
<table border="1">
  <tbody>
    <tr>
      <td><img src="notebookImgs/light-mask.png" alt="reference mask" style="width:640px;"><br /><b>Reference Mask</b></td>
      <td><img src="notebookImgs/sys-light.png" alt="system output mask" style="width:640px;"><br /><b>System Output Mask</b></td>
    </tr>
    <tr>
      <td><img src="notebookImgs/sys-light-weights.png" alt="no-score zone" style="width:640px;"><br /><b>No-Score Zone</b></td>
      <td><img src="notebookImgs/sys-light_colored.jpg" alt="color mask" style="width:640px;"><br /><b>Mask Scoring Example</b></td>
    </tr>
    <tr>
      <td>NIMBLE Mask Metric (NMM): -1.000000</td>
      <td><b>Total Pixels</b>: 6016000<br /><b>False Positives (FP)</b>: 175678<br /><b>False Negatives (FN)</b>: 462<br /><b>True Positives (TP)</b>: 11462<br /><b>No Score Zone (NS)</b>: 9161<br /></td>
    </tr>
  </tbody>
</table>

## Disclaimer

This software was developed at the National Institute of Standards
and Technology (NIST) by employees of the Federal Government in the
course of their official duties. Pursuant to Title 17 Section 105
of the United States Code, this software is not subject to copyright
protection and is in the public domain. NIST assumes no responsibility
whatsoever for use by other parties of its source code or open source
server, and makes no guarantees, expressed or implied, about its quality,
reliability, or any other characteristic.